Make sure you are in the right kernel (`regionalization`)!

In [ ]:
import bw2calc as bc
import bw2data as bd
import bw2regional as bwr
import bw2analyzer as ba
import csv
import geopandas as gpd
import pandas as pd
import numpy as np
import rasterio
from rasterio.plot import show

In [ ]:
bd.projects.set_current("LC IMPACT case study")

The keys for our functional units:

In [ ]:
for act in bd.Database("Fuel comparison"):
    print(act.key, act)

The methods we have available:

In [ ]:
[x for x in bd.methods if x[0] == 'LC-IMPACT'][:3]

In [ ]:
bwr.extension_tables

In [ ]:
show(rasterio.open(bwr.geocollections['sugarcane_water_intensity']['filepath']))

In [ ]:
show(rasterio.open(bwr.geocollections['sugarcane_landuse_intensity']['filepath']))

In [ ]:
show(rasterio.open(bwr.geocollections['weighted-pop-density']['filepath']))

Special activities that we might want to consider separately:

In [ ]:
irrigation = [x.id for x in bd.Database("ecoinvent") if 'irrigation' in x['name']]
crops = [x.id for x in bd.Database("ecoinvent") 
         if x['name'] in ('sugarcane production', 'maize grain production')]

In [ ]:
len(irrigation), len(crops)

In [ ]:
my_activity = bd.get_activity(id=irrigation[0])

In [ ]:
for exc in my_activity.biosphere():
    print(exc['amount'], exc.input, exc.output)

In [ ]:
activities = [
    ('Fuel comparison', 'sugarcane transport'),
    ('Fuel comparison', 'maize transport'),
    ('Fuel comparison', 'petrol transport'),
]

lcimpact = {
 ('LC-IMPACT', 'Land Use', 'Occupation', 'Marginal', 'Certain'): {
     'xt others': 'world-topo-ecoregions-weighted-pop-density',
     'activities': crops,
     'sugarcane': 'world-topo-ecoregions-sugarcane_landuse_intensity',
     'maize': 'world-topo-ecoregions-maize_landuse_intensity',
 },
 ('LC-IMPACT', 'Particulate Matter Formation', 'Marginal', 'All'): {
     'xt others': 'world-topo-particulate-matter-weighted-pop-density',
     'activities': [],
     'sugarcane': 'world-topo-particulate-matter-sugarcane_landuse_intensity',
     'maize': 'world-topo-particulate-matter-maize_landuse_intensity',
 },
 ('LC-IMPACT', 'Particulate Matter Formation', 'Marginal', 'Certain'): {
     'xt others': 'world-topo-particulate-matter-weighted-pop-density',
     'activities': [],
     'sugarcane': 'world-topo-particulate-matter-sugarcane_landuse_intensity',
     'maize': 'world-topo-particulate-matter-maize_landuse_intensity',
 },
 ('LC-IMPACT', 'Water Use', 'Ecosystem Quality', 'Surface Water', 'Marginal', 'All'): {
     'xt others': 'world-topo-watersheds-eq-sw-all-weighted-pop-density',
     'activities': irrigation + crops,
     'sugarcane': 'world-topo-watersheds-eq-sw-all-sugarcane_water_intensity',
     'maize': 'world-topo-watersheds-eq-sw-all-maize_water_intensity',
 },
 ('LC-IMPACT', 'Water Use', 'Ecosystem Quality', 'Surface Water', 'Marginal', 'Certain'): {
     'xt others': 'world-topo-watersheds-eq-sw-certain-weighted-pop-density',
     'activities': irrigation + crops,
     'sugarcane': 'world-topo-watersheds-eq-sw-certain-sugarcane_water_intensity',
     'maize': 'world-topo-watersheds-eq-sw-certain-maize_water_intensity',
 },
 ('LC-IMPACT', 'Water Use', 'Human Health', 'Marginal'): {
     'xt others': 'world-topo-watersheds-hh-weighted-pop-density',
     'activities': irrigation + crops,
     'sugarcane': 'world-topo-watersheds-hh-sugarcane_water_intensity',
     'maize': 'world-topo-watersheds-hh-maize_water_intensity',
 },
}

Let's do some calculations

In [ ]:
func_unit = {('Fuel comparison', 'sugarcane transport'): 1}
crop = 'sugarcane'
method = ('LC-IMPACT', 'Land Use', 'Occupation', 'Marginal', 'Certain')

In [ ]:
xt_ag = bwr.ExtensionTablesLCA(
    func_unit,
    method,
    xtable=lcimpact[method][crop],
    limitations={
        'activities': lcimpact[method]['activities'],
    },
)
xt_ag.lci()
xt_ag.lcia()
xt_ag.score

In [ ]:
xt_others = bwr.ExtensionTablesLCA(
    func_unit,
    method,
    xtable=lcimpact[method]['xt others'],
    limitations={
        'activities': lcimpact[method]['activities'],
        'activities mode': 'exclude'
    }
)
xt_others.lci()
xt_others.lcia()
xt_others.score

In [ ]:
xt_ag.characterized_inventory += xt_others.characterized_inventory

In [ ]:
xt_ag.geodataframe_inv_spatial_scale()

In [ ]:
xt_ag.geodataframe_ia_spatial_scale()

In [ ]:
xt_ag.geodataframe_ia_spatial_scale().explore(column='score_rel')